In [ ]:
import numpy as np
import pandas as pd
import os, sys
import anndata
from plotnine import *
import wandb 
import torch

sys.path.append('/code/decima/src/decima/')
from visualize import plot_gene_scatter
from read_hdf5 import extract_gene_data
from lightning import LightningModel

from grelu.transforms.prediction_transforms import Aggregate
from grelu.visualize import plot_attributions
from grelu.interpret.motifs import scan_sequences, compare_motifs
from grelu.sequence.format import strings_to_one_hot
from captum.attr import InputXGradient
import scanpy as sc

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

from grelu.io.motifs import read_meme_file
from tangermeme.plot import plot_pwm
hocomoco = read_meme_file('../H12CORE_meme_format.meme')

## Paths

In [ ]:
save_dir='/gstore/data/resbioai/grelu/decima/20240823'
matrix_file = os.path.join(save_dir, 'data.h5ad')
h5_file = os.path.join(save_dir, 'data.h5')

out_dir = os.path.join(save_dir, 'gwas_44traits')
pred_file = os.path.join(out_dir, 'gwas_variant_predictions_matched.h5ad')

pos_dir = os.path.join(out_dir, 'positive_variants')
neg_dir = os.path.join(out_dir, 'negative_variants')

trait_file = os.path.join(pos_dir, 'positive_variants_and_traits.csv')
matched_neg_file = os.path.join(neg_dir, 'negatives_matched.csv')

## Load Decima predictions

In [ ]:
ad = anndata.read_h5ad(matrix_file)

In [ ]:
ad.var['tss'] = ad.var.apply(lambda row: row.gene_start if row.strand=='+' else row.gene_end, axis=1)
ad.var.loc[ad.var.ensembl_canonical_tss.isna(), 'ensembl_canonical_tss'] = ad.var.loc[ad.var.ensembl_canonical_tss.isna(), 'tss']

In [ ]:
ad.obs['celltype_coarse'] = [row.celltype_coarse if row.dataset=='skin_atlas' else row.cell_type for row in ad.obs.itertuples()]

## Load variant predictions

In [ ]:
gwas_ = anndata.read_h5ad(pred_file)

In [ ]:
gwas_.var = gwas_.var.merge(ad.obs[['cell_type', 'celltype_coarse']].drop_duplicates(), left_index=True, right_on='cell_type').set_index('cell_type')

In [ ]:
gwas_.X = np.abs(gwas_.X)

In [ ]:
gwas_.obs.index = gwas_.obs_names = range(len(gwas_.obs))

In [ ]:
gwas = sc.get.aggregate(gwas_, by='celltype_coarse', func='mean', axis=1)
gwas.X = gwas.layers['mean']

In [ ]:
gwas_.X = gwas_.layers['gene_exp']

In [ ]:
exp = sc.get.aggregate(gwas_, by='celltype_coarse', func='mean', axis=1)

In [ ]:
gwas.layers['gene_exp'] = exp.layers['mean']

## Annotate Decima predictions

In [ ]:
gwas.obs['nearest_gene'] = gwas.obs.apply(lambda row: np.abs(row.pos - ad.var.loc[ad.var.chrom==row.chrom, 'tss']).sort_values().index[0], axis=1)
gwas.obs['nearest_canonical_gene'] = gwas.obs.apply(lambda row: np.abs(row.pos - ad.var.loc[ad.var.chrom==row.chrom, 'ensembl_canonical_tss']).sort_values().index[0], axis=1)

In [ ]:
gwas.obs['tss_dist'] = gwas.obs.apply(lambda row: row.pos - ad.var.loc[row.gene, 'tss'], axis=1)
gwas.obs['canonical_tss_dist'] = gwas.obs.apply(lambda row: row.pos - ad.var.loc[row.gene, 'ensembl_canonical_tss'], axis=1)

In [ ]:
gwas.obs['same'] = gwas.obs.gene.astype(str)==gwas.obs.nearest_gene.astype(str)
gwas.obs['same_canonical'] = gwas.obs.gene.astype(str)==gwas.obs.nearest_canonical_gene.astype(str)

In [ ]:
gwas.obs['gene_pearson'] = gwas.obs.gene.apply(lambda x: ad.var.loc[x, 'pearson']).astype(float)
gwas.obs['gene_sf_pearson'] = gwas.obs.gene.apply(lambda x: ad.var.loc[x, 'size_factor_pearson']).astype(float)

## Annotate positive variants

In [ ]:
match = pd.read_csv(matched_neg_file)
match = match.groupby(['pos_variant', 'gene']).variant.apply(list).reset_index()

traits = pd.read_csv(trait_file)

In [ ]:
traits = traits[['variant', 'trait_name', 'vep']].merge(match[['pos_variant', 'gene', 'variant']], left_on='variant', right_on='pos_variant').drop(columns='variant_x')
traits = traits.merge(gwas.obs[['variant', 'rsid', 'gene', 'pval', 'score', 'has_eqtl', 'is_best', 'same', 'same_canonical', 'gene_pearson', 'gene_sf_pearson', 'tss_dist', 'canonical_tss_dist']].rename(
    columns={'variant':'pos_variant'})).copy()
traits.head(2)

In [ ]:
traits['neg_score'] = traits.apply(lambda row:
    gwas.obs.loc[(gwas.obs.variant.isin(row.variant_y)) & (gwas.obs.gene==row.gene), 'score'].mean(), axis=1)
traits['delta'] = traits.apply(lambda row: (row.score - row.neg_score), axis=1)
traits.head(2)

## Load models

In [ ]:
wandb.login(host="https://genentech.wandb.io")
ckpts=[
'/gstore/data/resbioai/grelu/decima/20240823/lightning_logs/kugrjb50/checkpoints/epoch=3-step=2920.ckpt',
'/gstore/data/resbioai/grelu/decima/20240823/lightning_logs/i68hdsdk/checkpoints/epoch=2-step=2190.ckpt',
'/gstore/data/resbioai/grelu/decima/20240823/lightning_logs/0as9e8of/checkpoints/epoch=7-step=5840.ckpt',
'/gstore/data/resbioai/grelu/decima/20240823/lightning_logs/i9zsp4nm/checkpoints/epoch=8-step=6570.ckpt',
]
models = [LightningModel.load_from_checkpoint(ckpt).eval() for ckpt in ckpts]

## Add per-cell type deltas

In [ ]:
traits['deltas'] = traits.apply(
    lambda row: (gwas[gwas.obs.variant==row.pos_variant].X - gwas[(gwas.obs.variant.isin(row.variant_y)) & (gwas.obs.gene==row.gene)].X).mean(0),
    axis=1)
traits['delta_z'] = traits.deltas.apply(lambda x: (x - x.mean())/x.std())
traits['top5'] = traits.delta_z.apply(lambda x: gwas.var_names[np.argsort(x)[-5:]].tolist())
traits['ct_z2'] = traits.delta_z.apply(lambda x: gwas.var_names[x>2].tolist())

## Select top positive variants

In [ ]:
top = traits[(traits.pval < .01)].sort_values('delta', ascending=False)

## Examples

In [ ]:
def get_attrs(gene, ref_allele, alt_allele, rel_pos, top_cts, h5_file=h5_file, models=models):
    seq, mask = extract_gene_data(h5_file, gene, merge=False)
    tss_pos = np.where(mask[0] == 1)[0][0] - 2
    device = torch.device(0)
    ref_seq = seq.clone()
    alt_seq = seq.clone()

    ref_seq[:, rel_pos] = strings_to_one_hot(ref_allele).squeeze()
    alt_seq[:, rel_pos] = strings_to_one_hot(alt_allele).squeeze()

    tasks = ad.obs_names[ad.obs.cell_type.isin(top_cts)].tolist()
    ref_inputs = torch.vstack([ref_seq, mask]).to(device)
    alt_inputs = torch.vstack([alt_seq, mask]).to(device)

    attr_ref = []
    attr_alt = []
    
    for model in models:
        model.add_transform(Aggregate(tasks=tasks, task_aggfunc="mean", model=model))
        attributer = InputXGradient(model.to(device))
        with torch.no_grad():
            attr_ref.append(attributer.attribute(ref_inputs).cpu().numpy())
            attr_alt.append(attributer.attribute(alt_inputs).cpu().numpy())
    
    attr_ref = np.stack(attr_ref).mean(0)[:-1]
    attr_alt = np.stack(attr_alt).mean(0)[:-1]
    return attr_ref, attr_alt

## Individual examples

## rs138682554

https://www.genetics.opentargets.org/Variant/15_90884462_G_A/associations
eQTL: Artery tibial, Blood, Pancreas, Skin, Fibroblast, Macrophage, Monocyte

In [ ]:
rs='rs138682554'
gene = top[top.rsid==rs].gene.tolist()[0]
traits = top[top.rsid==rs].trait_name.tolist()[0]
top_cts = top[top.rsid==rs].ct_z2.tolist()[0]
rel_pos = gwas.obs[gwas.obs.rsid==rs].rel_pos.tolist()[0]
ref_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'ref_tx'].tolist()[0]
alt_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'alt_tx'].tolist()[0]

df = pd.DataFrame({
    'ct': gwas.var_names,
    'vep': top[top.rsid==rs].deltas.tolist()[0],
    'exp':np.array(gwas[gwas.obs.rsid==rs].layers['gene_exp']).squeeze()
})
df['z>2'] = df.ct.isin(top_cts)

print(gene)
print(traits)
print(top_cts)
print(gwas.obs.loc[gwas.obs.rsid==rs, 'tss_dist'].tolist()[0])
display(gwas.obs.loc[gwas.obs.rsid==rs])

In [ ]:
df['Cell Type'] = 'Other'
df.loc[df.ct.isin(['DC', 'Langerhans cell','conventional dendritic cell']), 'Cell Type'] = 'DC'
df.loc[df.ct.isin(['Macrophages', 'alveolar macrophage','macrophage']), 'Cell Type'] = 'Macrophages'
df.loc[df.ct.isin(['classical monocyte', 'intermediate monocyte','non-classical monocyte']), 'Cell Type'] = 'Monocytes'

In [ ]:
(
 ggplot(df, aes(x='exp', y='vep', color='Cell Type')) + geom_point() + theme_classic() 
    + theme(figure_size=(3.2,2.2)) 
    + ylab('Predicted absolute logFC\n(Background subtracted)') + xlab('Measured gene\n    expression')
    + scale_color_manual(values=['red', 'blue', 'orange', 'gray'])
)

In [ ]:
attr_ref, attr_alt = get_attrs(gene,ref_allele,alt_allele, rel_pos, top_cts)

In [ ]:
plot_attributions(attr_ref[:, rel_pos-10:rel_pos+10], figsize=(6,2), ylim=(-.55, 1.1),
                  highlight_positions=[10], alpha=.25)

In [ ]:
plot_attributions(attr_alt[:, rel_pos-10:rel_pos+10], figsize=(6,2), ylim=(-.55, 1.1), highlight_positions=[10],
                 alpha=.25)

In [ ]:
plot_pwm(hocomoco['SPI1.H12CORE.1.S.B'])

## rs8105903

https://www.genetics.opentargets.org/Variant/19_46784893_C_A/associations

eQTL: adipose, blood, thyyroid, tibial nerve, monocyte, fibroblast

In [ ]:
rs='rs8105903'
gene = top[top.rsid==rs].gene.tolist()[0]
traits = top[top.rsid==rs].trait_name.tolist()[0]
top_cts = top[top.rsid==rs].ct_z2.tolist()[0]
rel_pos = gwas.obs[gwas.obs.rsid==rs].rel_pos.tolist()[0]
ref_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'ref_tx'].tolist()[0]
alt_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'alt_tx'].tolist()[0]

df = pd.DataFrame({
    'ct': gwas.var_names,
    'vep': top[top.rsid==rs].deltas.tolist()[0],
    'exp':np.array(gwas[gwas.obs.rsid==rs].layers['gene_exp']).squeeze()
})
df['z>2'] = df.ct.isin(top_cts)

print(gene)
print(traits)
print(top_cts)
print(gwas.obs.loc[gwas.obs.rsid==rs, 'tss_dist'].tolist()[0], ref_allele, alt_allele)

In [ ]:
df=df.drop(index=40)

In [ ]:
df['Cell Type'] = 'Other'
df.loc[df.ct.isin(['Fibroblasts']), 'Cell Type'] = 'Fibroblasts'
df.loc[df.ct.isin(['Macrophages', 'alveolar macrophage','macrophage']), 'Cell Type'] = 'Macrophages'
df.loc[df.ct.isin(['classical monocyte', 'intermediate monocyte','non-classical monocyte']), 'Cell Type'] = 'Monocytes'

In [ ]:
(
 ggplot(df, aes(x='exp', y='vep', color='Cell Type')) + geom_point() + theme_classic() 
    + theme(figure_size=(3.2,2.2)) 
    + ylab('Predicted absolute logFC\n(Background subtracted)') 
    + xlab('Measured gene\n   expression')
    + scale_color_manual(['Red', 'Blue', 'Orange', 'Gray'])
)

In [ ]:
attr_ref, attr_alt = get_attrs(gene,ref_allele,alt_allele, rel_pos, ['Fibroblasts', 'fibroblast'])

In [ ]:
plot_attributions(attr_ref[:, rel_pos-10:rel_pos+10], figsize=(5.5,2), ylim=(-.17, .07),
                 highlight_positions=[10], alpha=.25)

In [ ]:
plot_attributions(attr_alt[:, rel_pos-10:rel_pos+10], figsize=(5.5,2), ylim=(-.17, .07),
                 highlight_positions=[10], alpha=.25)

In [ ]:
plot_pwm(hocomoco['ZEB2.H12CORE.0.P.B'])

In [ ]:
cmp = compare_motifs(ref_seq='TGTCCAGGGTATT', alt_seq='TGTCCAGGTATT', motifs='hocomoco_v12')
cmp

In [ ]:
pos_cts = ['Fibroblasts', 'fibroblast']

for tf in [x.split('.')[0] for x in cmp[cmp.alt > cmp.ref].motif]:
    if tf.startswith('ZN'):
        tf = tf.replace('ZN', 'ZNF')
    if tf.startswith('ZBT'):
        tf = tf.replace('ZBT', 'ZBTB')
    if tf in ad.var_names:
        tf_delta = ad[ad.obs.cell_type.isin(pos_cts), tf].X.mean() - ad[ad.obs.cell_type.isin(neg_cts), tf].X.mean()
        print(tf, tf_delta)
    else:
        print(tf)

https://www.genetics.opentargets.org/Variant/17_78187152_C_T/associations
eQTL: blood, brain, monocytes

In [ ]:
rs='rs144204502'
gene = top[top.rsid==rs].gene.tolist()[0]
traits = top[top.rsid==rs].trait_name.tolist()[0]
top_cts = top[top.rsid==rs].ct_z2.tolist()[0]
rel_pos = gwas.obs[gwas.obs.rsid==rs].rel_pos.tolist()[0]
ref_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'ref_tx'].tolist()[0]
alt_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'alt_tx'].tolist()[0]

df = pd.DataFrame({
    'ct': gwas.var_names,
    'vep': top[top.rsid==rs].deltas.tolist()[0],
    'exp':np.array(gwas[gwas.obs.rsid==rs].layers['gene_exp']).squeeze()
})
df['z>2'] = df.ct.isin(top_cts)

print(gene)
print(traits)
print(top_cts)
display(gwas.obs.loc[gwas.obs.rsid==rs])

In [ ]:
(
 ggplot(df, aes(x='exp', y='vep', color='z>2')) + geom_point() + theme_classic() 
    + theme(figure_size=(3,2)) + ylab('VEP') + xlab('Measured gene expression')
    +ggtitle(f'{rs}, {gene}')
)

In [ ]:
#attr_ref, attr_alt = get_attrs(gene,ref_allele,alt_allele, rel_pos, ['erythroid lineage cell'])
plot_attributions(attr_ref[:, rel_pos-15:rel_pos+15], figsize=(4,1), ylim=(-.4, .6))
plot_attributions(attr_alt[:, rel_pos-15:rel_pos+15], figsize=(4,1), ylim=(-.4, .6))

In [ ]:
cmp = compare_motifs(ref_seq='CCAAATCTCCCGCCAG', alt_seq='CCAAATCTCCCACCAG', motifs='hocomoco_v12', pthresh=.001)
cmp

In [ ]:
pos_cts = top_cts
neg_cts = df[(df.exp > 2.2) & (df.vep < .05)].ct.tolist()
print(neg_cts)

for tf in [x.split('.')[0] for x in cmp[cmp.ref > cmp.alt].motif]:
    if tf.startswith('ZN'):
        tf = tf.replace('ZN', 'ZNF')
    if tf.startswith('ZBT'):
        tf = tf.replace('ZBT', 'ZBTB')
    if tf in ad.var_names:
        tf_delta = ad[ad.obs.cell_type.isin(pos_cts), tf].X.mean() - ad[ad.obs.cell_type.isin(neg_cts), tf].X.mean()
        print(tf, tf_delta)
    else:
        print(tf)

GWAS catalog: missing

Opentargets: https://www.genetics.opentargets.org/Variant/2_10122732_G_A/associations

eQTL: None for RRM2, blood for other genes

In [ ]:
rs='rs72542787'
gene = top[top.rsid==rs].gene.tolist()[0]
traits = top[top.rsid==rs].trait_name.tolist()[0]
top_cts = top[top.rsid==rs].ct_z2.tolist()[0]
rel_pos = gwas.obs[gwas.obs.rsid==rs].rel_pos.tolist()[0]
ref_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'ref_tx'].tolist()[0]
alt_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'alt_tx'].tolist()[0]

df = pd.DataFrame({
    'ct': gwas.var_names,
    'vep': top[top.rsid==rs].deltas.tolist()[0],
    'exp':np.array(gwas[gwas.obs.rsid==rs].layers['gene_exp']).squeeze()
})
df['z>2'] = df.ct.isin(top_cts)

print(gene)
print(traits)
print(top_cts)
display(gwas.obs.loc[gwas.obs.rsid==rs])

In [ ]:
df['Cell Type'] = 'Other'
df.loc[df.ct.isin(['erythroid lineage cell']), 'Cell Type'] = 'erythroid lineage cell'
df.loc[df.ct.isin(['granulocyte']), 'Cell Type'] = 'granulocyte'

In [ ]:
(
 ggplot(df, aes(x='exp', y='vep', color='Cell Type')) + geom_point() + theme_classic() 
    + theme(figure_size=(3,2)) + ylab('VEP') + xlab('Measured gene expression')
    +ggtitle(f'{rs}, {gene}')
)

In [ ]:
df.sort_values('exp', ascending=False).head(5)

In [ ]:
attr_ref, attr_alt = get_attrs(gene,ref_allele,alt_allele, rel_pos, ['erythroid lineage cell'])
plot_attributions(attr_ref[:, rel_pos-15:rel_pos+25], figsize=(4,1))
plot_attributions(attr_alt[:, rel_pos-15:rel_pos+25], figsize=(4,1))

In [ ]:
cmp = compare_motifs(ref_seq='TGAGGGGGCGCCCGTGCACCCTGTCC', alt_seq='TGAGGGGACGCCCGTGCACCCTGTCC', motifs='hocomoco_v12')
cmp

In [ ]:
pos_cts = ['erythroid lineage cell']
neg_cts = df[(df.exp > 2) & (df.vep < .04)].ct.tolist()
print(neg_cts)

for tf in [x.split('.')[0] for x in cmp[cmp.alt > cmp.ref].motif]:
    if tf.startswith('ZN'):
        tf = tf.replace('ZN', 'ZNF')
    if tf.startswith('ZBT'):
        tf = tf.replace('ZBT', 'ZBTB')
    if tf in ad.var_names:
        tf_delta = ad[ad.obs.cell_type.isin(pos_cts), tf].X.mean() - ad[ad.obs.cell_type.isin(neg_cts), tf].X.mean()
        if tf_delta > 0.1:
            print(tf, tf_delta)
    else:
        print(tf)

In [ ]:
pos_cts = ['erythroid lineage cell']
neg_cts = df[(df.exp > 2) & (df.vep < .04)].ct.tolist()
print(neg_cts)

for tf in [x.split('.')[0] for x in cmp[cmp.alt < cmp.ref].motif]:
    if tf.startswith('ZN') and (~tf.startswith('ZNF')):
        tf = tf.replace('ZN', 'ZNF')
    if tf.startswith('ZBT'):
        tf = tf.replace('ZBT', 'ZBTB')
    if tf in ad.var_names:
        tf_delta = ad[ad.obs.cell_type.isin(pos_cts), tf].X.mean() - ad[ad.obs.cell_type.isin(neg_cts), tf].X.mean()
        if tf_delta > 0.1:
            print(tf, tf_delta)
    else:
        print(tf)

In [ ]:
plot_pwm(hocomoco['E2F4.H12CORE.2.S.B'])

In [ ]:
rs='rs117552144'
gene = top[top.rsid==rs].gene.tolist()[0]
traits = top[top.rsid==rs].trait_name.tolist()[0]
top_cts = top[top.rsid==rs].ct_z2.tolist()[0]
rel_pos = gwas.obs[gwas.obs.rsid==rs].rel_pos.tolist()[0]
ref_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'ref_tx'].tolist()[0]
alt_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'alt_tx'].tolist()[0]

df = pd.DataFrame({
    'ct': gwas.var_names,
    'vep': top[top.rsid==rs].deltas.tolist()[0],
    'exp':np.array(gwas[gwas.obs.rsid==rs].layers['gene_exp']).squeeze()
})
df['z>2'] = df.ct.isin(top_cts)

print(gene)
print(traits)
print(top_cts)
print(gwas.obs.loc[gwas.obs.rsid==rs, 'tss_dist'].tolist()[0], ref_allele, alt_allele)
display(gwas.obs.loc[gwas.obs.rsid==rs])

In [ ]:
df.sort_values('vep', ascending=False).head(6)

In [ ]:
df['Cell Type'] = df.ct.apply(lambda x: x if x in ['mucus secreting cell', 'club cell','goblet cell', 'lung secretory cell'] else 'Other')

In [ ]:
(
 ggplot(df, aes(x='exp', y='vep', color='Cell Type')) + geom_point() + theme_classic() 
    + theme(figure_size=(3.7,2.2)) + ylab('      Decima VEP score\n(Background subtracted)') 
    + xlab('Measured gene\n    expression')
    +ggtitle(f'{rs}, {gene}')
    +scale_color_manual(['gray', 'red', 'green', 'orange', 'blue'])
)

In [ ]:
attr_ref, attr_alt = get_attrs(gene,ref_allele,alt_allele, rel_pos, top_cts)
plot_attributions(attr_ref[:, rel_pos-10:rel_pos+10], figsize=(5.5,2),
                 alpha=.25, highlight_positions=[10])
plot_attributions(attr_alt[:, rel_pos-10:rel_pos+10], figsize=(5.5, 2),
                 alpha=.25, highlight_positions=[10])

In [ ]:
scan_sequences('GGGTTTTCCCCCC', motifs='hocomoco_v12', pthresh=.001)

In [ ]:
scan_sequences('GGGTTTTCCCTCC', motifs='hocomoco_v12', pthresh=.002)

In [ ]:
plot_pwm(hocomoco['NFKB1.H12CORE.0.PS.A'])

In [ ]:
pos_cts = ['mucus secreting cell', 'club cell','goblet cell', 'lung secretory cell']
neg_cts = df[(df.exp > 2) & (df.vep < .02)].ct.tolist()
print(neg_cts)

for tf in [x.split('.')[0] for x in cmp[cmp.ref > cmp.alt].motif]:
    if tf.startswith('ZN'):
        tf = tf.replace('ZN', 'ZNF')
    if tf.startswith('ZBT'):
        tf = tf.replace('ZBT', 'ZBTB')
    if tf in ad.var_names:
        tf_delta = ad[ad.obs.cell_type.isin(pos_cts), tf].X.mean() - ad[ad.obs.cell_type.isin(neg_cts), tf].X.mean()
        print(tf, tf_delta)
    else:
        print(tf)

eQTL: https://www.genetics.opentargets.org/Variant/3_196632439_G_A/associations

Tibial nerve

In [ ]:
rs='rs1684466'
gene = top[top.rsid==rs].gene.tolist()[0]
traits = top[top.rsid==rs].trait_name.tolist()[0]
top_cts = top[top.rsid==rs].ct_z2.tolist()[0]
rel_pos = gwas.obs[gwas.obs.rsid==rs].rel_pos.tolist()[0]
ref_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'ref_tx'].tolist()[0]
alt_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'alt_tx'].tolist()[0]

df = pd.DataFrame({
    'ct': gwas.var_names,
    'vep': top[top.rsid==rs].deltas.tolist()[0],
    'exp':np.array(gwas[gwas.obs.rsid==rs].layers['gene_exp']).squeeze()
})
df['z>2'] = df.ct.isin(top_cts)

print(gene)
print(traits)
print(top_cts)
display(gwas.obs.loc[gwas.obs.rsid==rs])

In [ ]:
(
 ggplot(df, aes(x='exp', y='vep', color='z>2')) + geom_point() + theme_classic() 
    + theme(figure_size=(3,2)) + ylab('VEP') + xlab('Measured gene expression')
    +ggtitle(f'{rs}, {gene}')
)

In [ ]:
df.sort_values('exp', ascending=False).head(10)

In [ ]:
attr_ref, attr_alt = get_attrs(gene,ref_allele,alt_allele, rel_pos, top_cts)
plot_attributions(attr_ref[:, rel_pos-15:rel_pos+15], figsize=(4,1))
plot_attributions(attr_alt[:, rel_pos-15:rel_pos+15], figsize=(4,1))

In [ ]:
cmp = compare_motifs(ref_seq='CCGCTCCCGGCCGGGAGTCGCT', alt_seq='CCGCTCCCGGCCGGAAGTCGCT', motifs='hocomoco_v12', pthresh=.001)
cmp

In [ ]:
pos_cts = top_cts
neg_cts = df[(df.exp > 2) & (df.vep < .02)].ct.tolist()
print(neg_cts)

for tf in [x.split('.')[0] for x in cmp[cmp.alt > cmp.ref].motif]:
    if tf.startswith('ZN'):
        tf = tf.replace('ZN', 'ZNF')
    if tf.startswith('ZBT'):
        tf = tf.replace('ZBT', 'ZBTB')
    if tf in ad.var_names:
        tf_delta = ad[ad.obs.cell_type.isin(pos_cts), tf].X.mean() - ad[ad.obs.cell_type.isin(neg_cts), tf].X.mean()
        if tf_delta>0.1:
            print(tf, tf_delta)
    else:
        print(tf)

In [ ]:
rs='rs12722502'
gene = top[top.rsid==rs].gene.tolist()[0]
traits = top[top.rsid==rs].trait_name.tolist()[0]
top_cts = top[top.rsid==rs].ct_z2.tolist()[0]
rel_pos = gwas.obs[gwas.obs.rsid==rs].rel_pos.tolist()[0]
ref_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'ref_tx'].tolist()[0]
alt_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'alt_tx'].tolist()[0]

df = pd.DataFrame({
    'ct': gwas.var_names,
    'vep': top[top.rsid==rs].deltas.tolist()[0],
    'exp':np.array(gwas[gwas.obs.rsid==rs].layers['gene_exp']).squeeze()
})
df['z>2'] = df.ct.isin(top_cts)

print(gene)
print(traits)
print(top_cts)
display(gwas.obs.loc[gwas.obs.rsid==rs])

In [ ]:
(
 ggplot(df, aes(x='exp', y='vep', color='z>2')) + geom_point() + theme_classic() 
    + theme(figure_size=(3,2)) + ylab('VEP') + xlab('Measured gene expression')
    +ggtitle(f'{rs}, {gene}')
)

In [ ]:
df.sort_values('exp', ascending=False).head(10)

In [ ]:
attr_ref, attr_alt = get_attrs(gene,ref_allele,alt_allele, rel_pos, ['CD4-positive, alpha-beta T cell', 'CD8-positive, alpha-beta T cell'])
plot_attributions(attr_ref[:, rel_pos-10:rel_pos+10], figsize=(5.5,2),
                 alpha=.25, highlight_positions=[10])
plot_attributions(attr_alt[:, rel_pos-10:rel_pos+10], figsize=(5.5, 2),
                 alpha=.25, highlight_positions=[10])

In [ ]:
rs='rs1049728'
gene = top[top.rsid==rs].gene.tolist()[0]
traits = top[top.rsid==rs].trait_name.tolist()[0]
top_cts = top[top.rsid==rs].ct_z2.tolist()[0]
rel_pos = gwas.obs[gwas.obs.rsid==rs].rel_pos.tolist()[0]
ref_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'ref_tx'].tolist()[0]
alt_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'alt_tx'].tolist()[0]

df = pd.DataFrame({
    'ct': gwas.var_names,
    'vep': top[top.rsid==rs].deltas.tolist()[0],
    'exp':np.array(gwas[gwas.obs.rsid==rs].layers['gene_exp']).squeeze()
})
df['z>2'] = df.ct.isin(top_cts)

print(gene)
print(traits)
print(top_cts)
display(gwas.obs.loc[gwas.obs.rsid==rs])

(
 ggplot(df, aes(x='exp', y='vep', color='z>2')) + geom_point() + theme_classic() 
    + theme(figure_size=(3,2)) + ylab('VEP') + xlab('Measured gene expression')
    +ggtitle(f'{rs}, {gene}')
)

In [ ]:
df.sort_values('vep', ascending=False).head(20)

In [ ]:
attr_ref, attr_alt = get_attrs(gene,ref_allele,alt_allele, rel_pos, ['enterocyte'])
plot_attributions(attr_ref[:, rel_pos-10:rel_pos+10], figsize=(5.5,2),
                 alpha=.25, highlight_positions=[10])
plot_attributions(attr_alt[:, rel_pos-10:rel_pos+10], figsize=(5.5, 2),
                 alpha=.25, highlight_positions=[10])

In [ ]:
cmp = compare_motifs(ref_seq='CACTTGGACTCTTGCTC', alt_seq='CACTTGGAGTCTTGCTC', motifs='hocomoco_v12', pthresh=.001)
cmp

In [ ]:
pos_cts = top_cts
neg_cts = df[(df.exp > 3) & (df.vep < .03)].ct.tolist()
print(neg_cts)

for tf in [x.split('.')[0] for x in cmp[cmp.ref > cmp.alt].motif]:
    if tf.startswith('ZN'):
        tf = tf.replace('ZN', 'ZNF')
    if tf.startswith('ZBT'):
        tf = tf.replace('ZBT', 'ZBTB')
    if tf in ad.var_names:
        tf_delta = ad[ad.obs.cell_type.isin(pos_cts), tf].X.mean() - ad[ad.obs.cell_type.isin(neg_cts), tf].X.mean()
        print(tf, tf_delta)
    else:
        print(tf)

In [ ]:
rs='rs11553699'
gene = top[top.rsid==rs].gene.tolist()[0]
traits = top[top.rsid==rs].trait_name.tolist()[0]
top_cts = top[top.rsid==rs].ct_z2.tolist()[0]
rel_pos = gwas.obs[gwas.obs.rsid==rs].rel_pos.tolist()[0]
ref_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'ref_tx'].tolist()[0]
alt_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'alt_tx'].tolist()[0]

df = pd.DataFrame({
    'ct': gwas.var_names,
    'vep': top[top.rsid==rs].deltas.tolist()[0],
    'exp':np.array(gwas[gwas.obs.rsid==rs].layers['gene_exp']).squeeze()
})
df['z>2'] = df.ct.isin(top_cts)

print(gene)
print(traits)
print(top_cts)
display(gwas.obs.loc[gwas.obs.rsid==rs])

In [ ]:
gwas.obs.loc[gwas.obs.rsid==rs, ['pos', 'strand', 'tss_dist', 'canonical_tss_dist', 'gene_end_', 'ensembl_canonical_tss_']]

In [ ]:
df['Cell Type'] = 'Other'
df.loc[df.ct.isin(['hematopoietic stem cell']), 'Cell Type'] = 'hematopoietic stem cell'
(
 ggplot(df, aes(x='exp', y='vep', color='Cell Type')) + geom_point() + theme_classic() 
    + theme(figure_size=(3,2)) + ylab('VEP') + xlab('Measured gene\nexpression')
    +ggtitle(f'{rs}, {gene}')
)

In [ ]:
attr_ref, attr_alt = get_attrs(gene,ref_allele,alt_allele, rel_pos, ['hematopoietic stem cell'])
plot_attributions(attr_ref[:, rel_pos-10:rel_pos+10], figsize=(5.5,2),
                 alpha=.25, highlight_positions=[10])
plot_attributions(attr_alt[:, rel_pos-10:rel_pos+10], figsize=(5.5, 2),
                 alpha=.25, highlight_positions=[10])

## rs79755767

In [ ]:
rs='rs79755767'
gene = top[top.rsid==rs].gene.tolist()[0]
traits = top[top.rsid==rs].trait_name.tolist()[0]
top_cts = top[top.rsid==rs].ct_z2.tolist()[0]
rel_pos = gwas.obs[gwas.obs.rsid==rs].rel_pos.tolist()[0]
ref_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'ref_tx'].tolist()[0]
alt_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'alt_tx'].tolist()[0]

df = pd.DataFrame({
    'ct': gwas.var_names,
    'vep': top[top.rsid==rs].deltas.tolist()[0],
    'exp':np.array(gwas[gwas.obs.rsid==rs].layers['gene_exp']).squeeze()
})
df['z>2'] = df.ct.isin(top_cts)

print(gene)
print(traits)
print(top_cts)
display(gwas.obs.loc[gwas.obs.rsid==rs])


In [ ]:
df['Cell Type'] = 'Other'
df.loc[df.ct.isin(['erythroid lineage cell']), 'Cell Type'] = 'Erythroid   \nlineage cell'
df.loc[df.ct.isin(['hematopoietic stem cell']), 'Cell Type'] = 'Hematopoietic\nstem cell        '
df.loc[df.ct.isin(['megakaryocyte']), 'Cell Type'] = 'Megakaryocyte         '

(
 ggplot(df, aes(x='exp', y='vep', color='Cell Type')) + geom_point() + theme_classic() 
    + theme(figure_size=(3.5,2.2)) 
    + ylab('Predicted absolute logFC\n(background subtracted)') 
    + xlab('Measured gene\n  expression')
    + scale_color_manual(['red', 'blue', 'orange', 'gray'])
)

In [ ]:
attr_ref, attr_alt = get_attrs(gene,ref_allele,alt_allele, rel_pos, ['hematopoietic stem cell', 'erythroid lineage cell'])
plot_attributions(attr_ref[:, rel_pos-10:rel_pos+10], figsize=(5.5,2),
                 alpha=.25, highlight_positions=[10], ylim=(-.15, .29))
plot_attributions(attr_alt[:, rel_pos-10:rel_pos+10], figsize=(5.5, 2),
                 alpha=.25, highlight_positions=[10], ylim=(-.15, .29))

In [ ]:
cmp = compare_motifs(ref_seq='CGGGGTAACCGCCCGGCT', alt_seq='CGGGGTAACTGCCCGGCT', motifs='hocomoco_v12')
cmp

In [ ]:
pos_cts = ['hematopoietic stem cell', 'erythroid lineage cell']
neg_cts = df[(df.exp > 2) & (df.vep < .04)].ct.tolist()
print(neg_cts)

for tf in [x.split('.')[0] for x in cmp[cmp.alt < cmp.ref].motif]:
    if tf.startswith('ZN') and (~tf.startswith('ZNF')):
        tf = tf.replace('ZN', 'ZNF')
    if tf.startswith('ZBT'):
        tf = tf.replace('ZBT', 'ZBTB')
    if tf in ad.var_names:
        tf_delta = ad[ad.obs.cell_type.isin(pos_cts), tf].X.mean() - ad[ad.obs.cell_type.isin(neg_cts), tf].X.mean()
        if tf_delta > 0.1:
            print(tf, tf_delta)
    else:
        print(tf)

In [ ]:
plot_pwm(hocomoco['RUNX2.H12CORE.1.S.B'])

No eQTL

In [ ]:
rs='rs74401481'
gene = top[top.rsid==rs].gene.tolist()[0]
traits = top[top.rsid==rs].trait_name.tolist()[0]
top_cts = top[top.rsid==rs].ct_z2.tolist()[0]
rel_pos = gwas.obs[gwas.obs.rsid==rs].rel_pos.tolist()[0]
ref_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'ref_tx'].tolist()[0]
alt_allele = gwas.obs.loc[gwas.obs.rsid==rs, 'alt_tx'].tolist()[0]

df = pd.DataFrame({
    'ct': gwas.var_names,
    'vep': top[top.rsid==rs].deltas.tolist()[0],
    'exp':np.array(gwas[gwas.obs.rsid==rs].layers['gene_exp']).squeeze()
})
df['z>2'] = df.ct.isin(top_cts)

print(gene)
print(traits)
print(top_cts)
print(ref_allele, alt_allele)
print(gwas.obs.loc[gwas.obs.rsid==rs, 'tss_dist'].tolist()[0])

display(gwas.obs[gwas.obs.rsid==rs][['variant', 'rsid', 'vep', 'gene', 'strand', 'gene_mask_start', 'tss_dist', 'has_eqtl', 'same', 'same_canonical']])

In [ ]:
ad.var.loc['KLF16']

In [ ]:
(
 ggplot(df, aes(x='exp', y='vep', color='z>2')) + geom_point() + theme_classic() 
    + theme(figure_size=(3,2)) + ylab('VEP') + xlab('Measured gene expression')
    +ggtitle(f'{rs}, {gene}')
)

In [ ]:
df.sort_values('exp', ascending=False).head(10)

In [ ]:
#attr_ref, attr_alt = get_attrs(gene,ref_allele,alt_allele, rel_pos, top_cts)
plot_attributions(attr_ref[:, rel_pos-15:rel_pos+15], figsize=(4,1), ylim=(-.05, .13))
plot_attributions(attr_alt[:, rel_pos-15:rel_pos+15], figsize=(4,1), ylim=(-.05, .13))

In [ ]:
cmp = compare_motifs(ref_seq='GGCTGGTTAGGGG', alt_seq='GGCTGATTAGGGG', motifs='hocomoco_v12', pthresh=.001)
cmp

In [ ]:
pos_cts = ['erythroid lineage cell']
neg_cts = df[(df.exp > 2) & (df.vep < .02)].ct.tolist()

for tf in [x.split('.')[0] for x in cmp[cmp.alt > cmp.ref].motif]:
    if tf.startswith('ZN'):
        tf = tf.replace('ZN', 'ZNF')
    if tf.startswith('ZBT'):
        tf = tf.replace('ZBT', 'ZBTB')
    if tf in ad.var_names:
        tf_delta = ad[ad.obs.cell_type.isin(pos_cts), tf].X.mean() - ad[ad.obs.cell_type.isin(neg_cts), tf].X.mean()
        if tf_delta > .1:
            print(tf, tf_delta)
    else:
        print(tf)

In [ ]:
plot_pwm(hocomoco['GATA1.H12CORE.1.PSM.A'])

In [ ]:
plot_pwm(hocomoco['GATA4.H12CORE.0.PSM.A'])